<a href="https://colab.research.google.com/github/maneeha/KGLLM/blob/main/Diabetes%20-%20Text%20to%20graph%20-%20RAG_With_Knowledge_graph(Neo4j)%20Excel%20and%20Rouge%20Evaluation%20Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# langchain-core

contains simple, core abstractions that have emerged as a standard, as well as LangChain Expression Language as a way to compose these components together. This package is now at version 0.1 and all breaking changes will be accompanied by a minor version bump.

# langchain-community
contains all third party integrations. We will work with partners on splitting key integrations out into standalone packages over the next month.

# langchain
contains higher-level and use-case specific chains, agents, and retrieval algorithms that are at the core of your application's cognitive architecture. We are targeting a launch of a stable 0.1 release for langchain in early January.#

In [1]:
%pip install --upgrade --quiet  langchain langchain-community langchain-openai langchain-experimental neo4j wikipedia tiktoken yfiles_jupyter_graphs

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.3/204.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.6/293.6 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.8/389.8 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.3/361.3 kB 12.9 MB/s eta 0:00:00
   ━━━

In [2]:
from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

In [4]:
from google.colab import userdata
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')

In [5]:
from typing import Tuple, List, Optional

In [6]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser

In [7]:
from langchain_core.runnables import ConfigurableField

In [8]:
from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase


In [9]:
import os

In [10]:
try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass

In [11]:
from langchain_community.vectorstores import Neo4jVector

In [12]:
from google.colab import userdata
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')

In [13]:
NEO4J_URI="neo4j+s://2ff1ce5d.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="jaCuHWs8uM_CmCjPhp99hukq8pwbt-ud5B9pZ-qIjtQ"

In [14]:
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["NEO4J_URI"] = NEO4J_URI
os.environ["NEO4J_USERNAME"] = NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"] = NEO4J_PASSWORD

In [15]:
from langchain_community.graphs import Neo4jGraph

In [16]:
graph = Neo4jGraph()

In [17]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [18]:
from langchain.document_loaders import TextLoader

In [19]:
file_paths = ["/content/drive/MyDrive/refined/20121CoffmanNortonBeene.txt", "/content/drive/MyDrive/refined/insulin.txt", "/content/drive/MyDrive/refined/Blood gulocose monitoring.txt", "/content/drive/MyDrive/refined/blood gulocose.txt", "/content/drive/MyDrive/refined/complications.txt", "/content/drive/MyDrive/refined/diabetes awareness_TXT.txt", "/content/drive/MyDrive/refined/diabetes exercise.txt", "/content/drive/MyDrive/refined/diabetes in pregnancy.txt", "/content/drive/MyDrive/refined/Diabetes medication_TXT.txt" , "/content/drive/MyDrive/refined/diabetes monitoring_TXT.txt" , "/content/drive/MyDrive/refined/Diabetes Nutrition.txt", "/content/drive/MyDrive/refined/diabetes stress mgmt_TXT.txt" , "/content/drive/MyDrive/refined/Diabetes symptoms.txt", "/content/drive/MyDrive/refined/Diabetes tech_TXT.txt", "/content/drive/MyDrive/refined/Effectiveness of diabetes education and awareness of diabetes mellitus  in  combating  diabetes  in  the  United  Kigdom;  a  literature review .txt", "/content/drive/MyDrive/refined/gulocose monitoring.txt", "/content/drive/MyDrive/refined/KnowledgeofDMgeneralpublicKL.txt", "/content/drive/MyDrive/refined/Public knowledge and awareness of diabetes mellitus, its risk factors, complications, and prevention methods among adults in Poland—A 2022 nationwide cross-sectional survey _ Enhanced Reader.txt", "/content/drive/MyDrive/refined/Type 1 diabetes.txt"]  # Add your file paths here
raw_documents = []
for file_path in file_paths:
    loader = TextLoader(file_path)
    raw_documents.extend(loader.load())


In [ ]:
#from langchain.document_loaders import WikipediaLoader
#raw_documents = WikipediaLoader(query="Diabetes_management").load()

In [20]:
len(raw_documents)

19

In [ ]:
#raw_documents[:3]

In [21]:
from langchain.text_splitter import TokenTextSplitter
text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)
documents = text_splitter.split_documents(raw_documents)

In [22]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-0125")

In [ ]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
llm_transformer = LLMGraphTransformer(llm=llm)

In [ ]:
graph_documents = llm_transformer.convert_to_graph_documents(documents)

In [23]:
graph_documents

In [ ]:
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

In [24]:
import pandas as pd


In [25]:
graph

In [26]:
# directly show the graph resulting from the given Cypher query
default_cypher = "MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t LIMIT 7000"
default_cypher

'MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t LIMIT 7000'

In [27]:
from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase

In [28]:
try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass

In [29]:
def showGraph(cypher: str = default_cypher):
    # create a neo4j session to run queries
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))
    session = driver.session()
    widget = GraphWidget(graph = session.run(cypher).graph())
    widget.node_label_mapping = 'id'
    display(widget)
    return widget

In [ ]:
showGraph()

GraphWidget(layout=Layout(height='800px', width='100%'))

GraphWidget(layout=Layout(height='800px', width='100%'))

In [30]:
from typing import Tuple, List, Optional

In [31]:
from langchain_community.vectorstores import Neo4jVector

In [32]:
from langchain_openai import OpenAIEmbeddings
vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(),
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

In [33]:
graph.query("CREATE FULLTEXT INDEX entity IF NOT EXISTS FOR (e:__Entity__) ON EACH [e.id]")

[]

In [35]:
from langchain_core.pydantic_v1 import BaseModel, Field
# Extract entities from text
class Entities(BaseModel):
    """Identifying information about entities."""

    names: List[str] = Field(
        ...,
        description="All the diabetes related entities that "
        "appear in the text",
    )


In [36]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

In [37]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting diabetes symptoms diabetes medication diabetes exercise diabetes insulin diabetes technology diabetes nutrition diabetes types diabetes management diabetes prevention diabetes cure and diabetes care related entities from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)

In [38]:
entity_chain = prompt | llm.with_structured_output(Entities)

In [39]:
entity_chain.invoke({"question": "At what age diabetes risk gets increased?"}).names

['diabetes risk']

In [40]:
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars

In [41]:
def generate_full_text_query(input: str) -> str:
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    return full_text_query.strip()


In [42]:
# Fulltext index query
def structured_retriever(question: str) -> str:
    result = ""
    entities = entity_chain.invoke({"question": question})
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('entity', $query, {limit:10})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 7000
            """,
            {"query": generate_full_text_query(entity)},
        )
        result += "\n".join([el['output'] for el in response])
    return result

In [43]:
print(structured_retriever("What Diabetes is?"))

Acta Diabetol - PUBLISHED_ARTICLE -> Narlidere Health District
Diabet Med. - PUBLISHED_IN -> Public Awareness
Insulin Treatment - PUBLISHED_IN -> Diabet Med
Diabetes - ASSOCIATED_WITH -> Diabetic Cardiomyopathy
Diabetic Adults - PUBLISHED_IN -> Dodd, M.
Diabetic Adults - PUBLISHED_IN -> Doty, M. M.
Diabetic Adults - PUBLISHED_IN -> Garcia, A. A.
Diabetic Adults - PUBLICATION -> Dodd, M.
Diabetic Adults - PUBLICATION -> Doty, M. M.
Diabetic Adults - PUBLICATION -> Garcia, A. A.
Diabetic Retinopathy - AFFECTS -> United States
Diabetic Retinopathy - AFFECTS -> Peripheral Retina
Diabetic Retinopathy - AFFECTS -> Macula
Diabetic Retinopathy - USED_FOR -> Panretinal Laser Photocoagulation
Diabetic Retinopathy - USED_FOR -> Vegf Inhibitors
Diabetic Retinopathy - ASSOCIATED_WITH -> Hypertension
Diabetic Retinopathy - ASSOCIATED_WITH -> Insulin Resistance
Diabetic Retinopathy - ASSOCIATED_WITH -> Body Mass Index
Diabetic Retinopathy - ASSOCIATED_WITH -> Tobacco Use
Diabetic Retinopathy - ASSOCI

In [44]:
def retriever(question: str):
    print(f"Search query: {question}")
    structured_data = structured_retriever(question)
    unstructured_data = [el.page_content for el in vector_index.similarity_search(question)]
    final_data = f"""Structured data:
{structured_data}
Unstructured data:
{"#Document ". join(unstructured_data)}
    """
    return final_data

In [45]:
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question,
in its original language.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""

In [46]:
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

In [47]:
def _format_chat_history(chat_history: List[Tuple[str, str]]) -> List:
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer

In [48]:
_search_query = RunnableBranch(
    # If input includes chat_history, we condense it with the follow-up question
    (
        RunnableLambda(lambda x: bool(x.get("chat_history"))).with_config(
            run_name="HasChatHistoryCheck"
        ),  # Condense follow-up question and chat into a standalone_question
        RunnablePassthrough.assign(
            chat_history=lambda x: _format_chat_history(x["chat_history"])
        )
        | CONDENSE_QUESTION_PROMPT
        | ChatOpenAI(temperature=0)
        | StrOutputParser(),
    ),
    # Else, we have no chat history, so just pass through the question
    RunnableLambda(lambda x : x["question"]),
)

In [49]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
Use natural language and be concise.
Answer:"""

In [50]:
prompt = ChatPromptTemplate.from_template(template)

In [51]:
chain = (
    RunnableParallel(
        {
            "context": _search_query | retriever,
            "question": RunnablePassthrough(),
        }
    )
    | prompt
    | llm
    | StrOutputParser()
)

In [52]:
chain.invoke({"question": "how stress-related hormones affects diabetes and sugar level?"})

Search query: how stress-related hormones affects diabetes and sugar level?


'Stress-related hormones can affect diabetes and blood sugar levels by causing insulin resistance, leading to higher blood sugar levels. Stress can also trigger the release of glucose into the bloodstream, further raising blood sugar levels. Managing stress is important in controlling diabetes and maintaining stable blood sugar levels.'

In [53]:
chain.invoke(
    {
        "question": "When diabetes gets worst",
        "chat_history": [("how stress-related hormones affects diabetes and sugar level?", "increases diabetes")],
    }
)

Search query: When does diabetes get worse?


"When diabetes worsens, it can lead to serious complications such as heart disease, nerve damage, kidney damage, eye damage, foot complications, skin problems, hearing impairment, Alzheimer's disease, and depression."

In [54]:
#import pandas as pd
#from pydantic import BaseModel, Field
#from typing import List
#from langchain_openai import ChatOpenAI
#from langchain.prompts import ChatPromptTemplate
#from langchain_experimental.graph_transformers import LLMGraphTransformer
#from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars


# Assuming RunnableParallel and StrOutputParser exist and are properly imported
chain = (
    RunnableParallel(
        {
            "context":  _search_query | retriever,
            "question": RunnablePassthrough(),
        }
    )
    | prompt
    | llm
    | StrOutputParser()
)

# Example to test with a single question
result = chain.invoke({"question": "how stress-related hormones affects diabetes and sugar level?"})
print(result)



Search query: how stress-related hormones affects diabetes and sugar level?
Stress-related hormones can affect diabetes and blood sugar levels by causing insulin resistance, leading to higher blood sugar levels. Stress can also trigger the release of glucose into the bloodstream, further raising blood sugar levels. Managing stress is important in controlling diabetes and maintaining stable blood sugar levels.


In [55]:
# Read questions from the Excel file
questions_df = pd.read_excel('/content/drive/MyDrive/refined/RAGTest.xlsx')  # Replace with your file path
questions = questions_df['Question'].tolist()

# Prepare a list to store generated answers
generated_answers = []

# Iterate through questions and generate answers
for question in questions:
    answer = chain.invoke({"question": question})
    generated_answers.append(answer)

# Write generated answers back to the Excel file
questions_df['Generated Answer'] = generated_answers
questions_df.to_excel('/content/drive/MyDrive/refined/questions_with_answers.xlsx', index=False)

# Optionally, compare generated answers with human-generated responses
if 'Human Answer' in questions_df.columns:
    human_answers = questions_df['Human Answer'].tolist()
    questions_df['Accuracy'] = questions_df.apply(
        lambda row: 1 if row['Generated Answer'].strip() == row['Human Answer'].strip() else 0,
        axis=1
    )
    accuracy = questions_df['Accuracy'].mean()
    print(f"Accuracy of generated answers: {accuracy * 100:.2f}%")
else:
    print("Human Answer column not found in the Excel file.")

Search query: What are some early warning signs of diabetes that often go unnoticed?
Search query: How can intermittent fasting affect blood sugar levels in diabetic patients?
Search query: What role does sleep quality play in the management of Type 2 diabetes?
Search query: How can diabetic patients incorporate mindful eating practices to better manage their condition?
Search query: What are the benefits of using a hybrid closed-loop insulin pump system?
Search query: How does stress impact blood glucose levels, and what are effective stress management techniques for diabetics?
Search query: Can consuming probiotics and prebiotics improve insulin sensitivity in diabetic patients?
Search query: What are some innovative approaches to diabetes education that have shown promise in recent studies?
Search query: How can a diabetic patient safely participate in endurance sports or high-intensity interval training (HIIT)?
Search query: What are the latest advancements in artificial pancreas s

In [56]:
questions_df['Generated Answer'][0]

"Some early warning signs of diabetes that often go unnoticed include increased hunger, increased thirst, weight loss, frequent urination, blurry vision, extreme fatigue, and sores that don't heal."

In [57]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=214b8fedd3d38162455cdfedb54946ae42f6f91830fc44e9fc0f0b04f9a0f0b0
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [58]:
!pip install sacrebleu
!pip install nltk


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 977.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 1.8 MB/s eta 0:00:00


In [59]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')  # This is for wordnet's morphological processing, required by METEOR


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [61]:
import pandas as pd
import nltk
from rouge_score import rouge_scorer
from transformers import BertTokenizer, BertForSequenceClassification, pipeline
import torch

# Ensure NLTK resources are downloaded
nltk.download('punkt')

# Load the Excel sheet
file_path = '/content/drive/MyDrive/refined/questions_with_answers.xlsx'  # Replace with your file path
df = pd.read_excel(file_path)

# Initialize evaluation tools
rouge = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

# Create a pipeline for BERTScore (alternatively, use other models like roberta)
bert_scorer = pipeline("feature-extraction", model=model, tokenizer=tokenizer, return_tensors="pt")

# Function to calculate BERTScore
def compute_bertscore(human_resp, generated_resp):
    human_emb = bert_scorer(human_resp)[0].mean(dim=0)
    generated_emb = bert_scorer(generated_resp)[0].mean(dim=0)
    similarity = torch.cosine_similarity(human_emb, generated_emb, dim=0).item()
    return similarity

# Evaluate the responses
evaluation_results = []
for index, row in df.iterrows():
    question = row['Question']
    human_resp = row['Human Answer']
    generated_resp = row['Generated Answer']

    # Calculate ROUGE scores
    rouge_scores = rouge.score(human_resp, generated_resp)
    rouge1_f1 = rouge_scores['rouge1'].fmeasure
    rougeL_f1 = rouge_scores['rougeL'].fmeasure

    # Calculate BERTScore
    bert_score = compute_bertscore(human_resp, generated_resp)

    # Store the evaluation results
    evaluation_results.append({
        "Question": question,
        "Human Response": human_resp,
        "Generated Response": generated_resp,
        "ROUGE-1 F1": rouge1_f1,
        "ROUGE-L F1": rougeL_f1,
        "BERTScore": bert_score
    })

# Convert the results to a DataFrame
eval_df = pd.DataFrame(evaluation_results)

# Save the evaluation results to a new Excel sheet
eval_df.to_excel('/content/drive/MyDrive/refined/evaluation_results.xlsx', index=False)  # Replace with your desired save path

# Optionally, print the evaluation results
print(eval_df)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


                                             Question  \
0   What are some early warning signs of diabetes ...   
1   How can intermittent fasting affect blood suga...   
2   What role does sleep quality play in the manag...   
3   How can diabetic patients incorporate mindful ...   
4   What are the benefits of using a hybrid closed...   
..                                                ...   
94  How can diabetes management apps help in contr...   
95  What are the benefits of using wearable device...   
96  How can I prepare for a diabetes-related emerg...   
97  What is diabetic ketoacidosis, and how can it ...   
98                How does diabetes affect pregnancy?   

                                       Human Response  \
0   Early warning signs of diabetes that are often...   
1   Intermittent fasting can help some diabetic pa...   
2   Poor sleep quality can negatively impact blood...   
3   Mindful eating encourages patients to pay atte...   
4   A hybrid closed-loop insul

In [69]:
# Step 1: Install required libraries
!pip install sacrebleu nltk pandas openpyxl

import pandas as pd
import sacrebleu
from nltk.translate import meteor_score
from nltk.tokenize import word_tokenize
import nltk

# Download NLTK tokenizer
nltk.download('punkt')

# Step 2: Mount Google Drive to access the Excel file
from google.colab import drive
drive.mount('/content/drive')

# File path to your Excel sheet
file_path = '/content/drive/MyDrive/refined/questions_with_answers.xlsx'

# Step 3: Load the Excel sheet
df = pd.read_excel(file_path)

# Ensure that your DataFrame contains the correct columns
# Print the first few rows to verify
print(df.head())

# Step 4: Define functions for tokenization and scoring
def tokenize_text(text):
    return word_tokenize(text.lower())  # Tokenize and lower case the text

def calculate_bleu_score(hypothesis, references):
    # References should be a list of lists of tokens
    return sacrebleu.corpus_bleu(hypothesis, references).score

def calculate_meteor_score(reference, hypothesis):
    return meteor_score.meteor_score(reference, hypothesis)

# Step 5: Initialize lists to store results
results = []

# Step 6: Calculate scores for each row in the DataFrame
for index, row in df.iterrows():
    question = row['Question']
    human_response = row['Human Answer']
    generated_response = row['Generated Answer']

    # Tokenize responses
    human_tokens = tokenize_text(human_response)
    generated_tokens = tokenize_text(generated_response)

    # For BLEU score calculation
    references = human_tokens  # List of references, each being a list of tokens
    hypothesis = generated_tokens

    # Calculate BLEU and METEOR scores
    bleu_score = calculate_bleu_score(hypothesis, references)
    meteor_score_val = calculate_meteor_score(human_response, generated_response)

    # Store results
    results.append({
        "Question": question,
        "Human Response": human_response,
        "Generated Response": generated_response,
        "BLEU Score": bleu_score,
        "METEOR Score": meteor_score_val
    })

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

# Save the results to an Excel file
results_file_path = '/content/drive/MyDrive/refined/evaluation_results.xlsx'
results_df.to_excel(results_file_path, index=False)

# Print the results
print(results_df)
